In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install simplet5 -q

In [ ]:
df=pd.read_csv("/kaggle/input/news-summary/news_summary.csv",encoding='latin-1',usecols=['headlines','text'])

In [ ]:
df.head()

**T5 Data Prep with Training Data Column Names - source_text & target_text**

In [ ]:
#simpleT5 expects Dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"headlines":"target_text","text":"source_text"})
df = df[['source_text','target_text']]

In [ ]:
df.head()

***T5 Data Prep with Summarization Tax Prefix***

In [ ]:
df['source_text']= 'summarize: '+df['source_text']
df

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3)
train_df.shape, test_df.shape

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

In [ ]:
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=5, use_gpu=True)

In [ ]:
!( cd outputs; ls )

In [ ]:
model.load_model("t5","outputs/simplet5-epoch-2-train-loss-0.9396-val-loss-1.2222", use_gpu=True)

In [ ]:
text_to_summarize="""summarize: Twitter’s interim resident grievance officer for India has stepped down, leaving the micro-blogging site without a grievance official as mandated by the new IT rules to address complaints from Indian subscribers, according to a source.

The source said that Dharmendra Chatur, who was recently appointed as interim resident grievance officer for India by Twitter, has quit from the post.

The social media company’s website no longer displays his name, as required under Information Technology (Intermediary Guidelines and Digital Media Ethics Code) Rules 2021.

Twitter declined to comment on the development.

The development comes at a time when the micro-blogging platform has been engaged in a tussle with the Indian government over the new social media rules. The government has slammed Twitter for deliberate defiance and failure to comply with the country’s new IT rules.
"""
model.predict(text_to_summarize)